In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [3]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [4]:
# current_date = pd.to_datetime(datetime.utcnow()).floor('H')
current_date = pd.Timestamp('2024-05-08 23:00:00')
print(f'{current_date=}')

current_date=Timestamp('2024-05-08 23:00:00')


In [5]:
from src.inference import load_batch_of_features_from_feature_store

In [6]:
features = load_batch_of_features_from_feature_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/699541
Connected. Call `.close()` to terminate connection gracefully.
2024-05-10 10:42:16,979 INFO: Feature view timeseries_hourly_feature_view already exists
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/699541
Connected. Call `.close()` to terminate connection gracefully.
2024-05-10 10:42:25,698 WARNING: DeprecationWarning: DatetimeTZBlock is deprecated and will be removed in a future version. Use public APIs instead.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.88s) 


In [7]:
from src.inference import (load_model_from_registry, get_model_predictions)

In [8]:
model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/699541
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,0.0,2024-05-08 23:00:00
1,2,0.0,2024-05-08 23:00:00
2,3,0.0,2024-05-08 23:00:00
3,4,5.0,2024-05-08 23:00:00
4,5,0.0,2024-05-08 23:00:00
...,...,...,...
260,261,19.0,2024-05-08 23:00:00
261,262,14.0,2024-05-08 23:00:00
262,263,51.0,2024-05-08 23:00:00
263,264,49.0,2024-05-08 23:00:00


In [10]:
from src.feature_store_api import get_feature_store
import src.config as config

In [11]:
# 
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=config.FEATURE_GROUP_MODEL_PREDICTIONS_VERSION,
    description='Predictions generated by production model',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour'
    )

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/699541
Connected. Call `.close()` to terminate connection gracefully.


In [12]:
feature_group.insert(predictions, write_options={'wait_for_job': False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/699541/fs/695364/fg/800117


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/699541/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x140568e50>, None)